# Getting Started with Farm Notebooks

Welcome! This notebook will help you get set up with all the tools you need to start analyzing your flock data using NSIP (National Sheep Improvement Program) tools.

**What is a Jupyter Notebook?** Think of it as a living document that mixes explanations (like this text) with runnable code. You read the explanations, then run the code to see results -- right here on your screen.

**Two kinds of cells:**
- **Markdown cells** (like this one) contain plain text, headings, and instructions.
- **Code cells** (the gray boxes below) contain Python code you can run.

**How to run a cell:** Click on it, then press **Shift + Enter**. The code runs and the result appears below it. Try it on the cells below!

## How to Run This Notebook

Follow these steps to get started:

1. Make sure you have Python 3.12 or newer installed on your computer.
2. Install the `uv` package manager (instructions below).
3. Run `uv sync --extra data` in your terminal to install all dependencies.
4. Start Jupyter with `uv run jupyter notebook` or `uv run jupyter lab`.
5. Open this notebook file (`getting-started.ipynb`) in Jupyter.
6. Work through each cell from top to bottom, pressing **Shift + Enter** to run each one.

If a code cell shows an error, read the message carefully -- it usually tells you what is missing or what went wrong.

## Installing uv and Project Dependencies

The `uv` tool manages Python packages for this project. It keeps everything organized so different projects do not conflict with each other.

If you have not installed `uv` yet, run the following cell. If you already have it, skip to the dependency install step.

In [ ]:
# Install uv (only needed once -- skip if you already have it)
# Uncomment the line below to install:
# !curl -LsSf https://astral.sh/uv/install.sh | sh

Now install all the project dependencies, including the data-science extras (pandas, numpy, matplotlib).

In [ ]:
# Install project dependencies with data science extras
!uv sync --extra data

## Installing the NSIP CLI

The NSIP CLI is a command-line tool that lets you search for animals, look up EBVs (Estimated Breeding Values), check inbreeding coefficients, and more -- all from your computer.

Download the correct binary for your operating system:

| Operating System | Download Link |
|-----------------|---------------|
| macOS (Apple Silicon) | [nsip-darwin-arm64](https://github.com/zircote/nsip/releases/latest) |
| macOS (Intel) | [nsip-darwin-amd64](https://github.com/zircote/nsip/releases/latest) |
| Linux | [nsip-linux-amd64](https://github.com/zircote/nsip/releases/latest) |
| Windows | [nsip-windows-amd64.exe](https://github.com/zircote/nsip/releases/latest) |

After downloading, place the binary somewhere on your system PATH (for example, `/usr/local/bin/` on macOS/Linux).

Let us verify the installation:

In [ ]:
# Check that the NSIP CLI is installed and show the version
!nsip --version

Let us also check when the NSIP data was last updated. This tells you how fresh the genetic information is.

In [ ]:
# Check the date the NSIP data was last updated
!nsip date-updated

## Docker Setup for NSIP MCP Server

If you plan to use the NSIP MCP (Model Context Protocol) server -- for example, to connect NSIP data to Claude Desktop -- you will need Docker installed.

**What is Docker?** It is a tool that runs software in a self-contained package (called a container) so you do not have to install everything manually. Think of it like a shipping container for software.

First, pull the NSIP MCP server image:

In [ ]:
# Pull the latest NSIP MCP server Docker image
!docker pull ghcr.io/zircote/nsip:latest

To run the MCP server manually (useful for testing), you would use this command in your terminal:

```bash
docker run --rm -i ghcr.io/zircote/nsip:latest
```

The server communicates over standard input/output, so it is typically launched by an MCP client (like Claude Desktop) rather than run directly.

## Quick Test: Your First Sheep Search

Let us make sure everything is working by searching for Katahdin sheep with the name "storm" in the NSIP database. Katahdin is breed ID 640.

This cell runs the NSIP CLI from Python and parses the results.

In [ ]:
import json  # noqa: E402  (tells the linter this import position is intentional)
import subprocess  # noqa: E402  (tells the linter this import position is intentional)

# Search for Katahdin sheep named "storm"
result = subprocess.run(
    ["nsip", "--json", "search", "--breed-id", "640", "--name", "storm"],
    capture_output=True,
    text=True,
)

if result.returncode == 0:
    raw_search_results = json.loads(result.stdout)
    print(f"Found {len(raw_search_results)} animals.")
    print(json.dumps(raw_search_results[:3], indent=2))
else:
    print(f"Error: {result.stderr}")

If you see a list of animals above, congratulations -- your setup is working! If you see an error, double-check that the `nsip` binary is installed and on your system PATH.

## Configuring MCP for Claude Desktop

To use NSIP data directly inside Claude Desktop (so you can ask Claude questions about your sheep), you need to add an MCP server configuration.

### Option 1: Using Docker (Recommended)

Add this to your Claude Desktop MCP configuration file:

```json
{
  "mcpServers": {
    "nsip": {
      "command": "docker",
      "args": ["run", "--rm", "-i", "ghcr.io/zircote/nsip:latest"]
    }
  }
}
```

### Option 2: Using the CLI Binary Directly

If you prefer not to use Docker, point directly to the `nsip` binary:

```json
{
  "mcpServers": {
    "nsip": {
      "command": "nsip",
      "args": ["mcp"]
    }
  }
}
```

### Where to Put This Configuration

- **macOS**: `~/Library/Application Support/Claude/claude_desktop_config.json`
- **Windows**: `%APPDATA%\Claude\claude_desktop_config.json`

After saving, restart Claude Desktop. You should see NSIP listed as an available tool.

## Conclusion

You now have everything set up to start working with NSIP sheep genetics data. Here is a checklist of what you should have ready:

- [x] Understand how Jupyter notebooks work (markdown and code cells)
- [x] `uv` installed and project dependencies synced
- [x] NSIP CLI installed and verified
- [x] Docker image pulled for the MCP server (optional)
- [x] First search completed successfully
- [x] MCP configuration ready for Claude Desktop (optional)

### Next Steps

- [ ] Open `nsip-sheep-genetics.ipynb` to learn about EBVs and how to analyze your flock
- [ ] Try searching for animals from your own flock
- [ ] Set up Claude Desktop with the MCP server for conversational flock analysis